In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(r'/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv(r'/kaggle/input/digit-recognizer/test.csv')
df.head()

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split

We are going to split data with test size 20%

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('label', axis = 1), df['label'], test_size = 0.2)

In [ ]:
# Let's print the shape of data after train test split
x_train.shape, x_test.shape

* To pass to our model we need to change the shape of our dataset

In [ ]:
x_train = np.array(x_train).reshape(x_train.shape[0], 28, 28, 1)
x_test = np.array(x_test).reshape(x_test.shape[0], 28, 28, 1)

test_array =  np.array(test).reshape(test.shape[0], 28, 28, 1)

In [ ]:
x_train = x_train / 255.0

x_test = x_test / 255.0

test_array = test_array / 255.0

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
test_array = test_array.astype('float32')

In [ ]:
#Convert class vector to binary class matrices, It's simply like a one hot encoding
y_train = keras.utils.to_categorical(y_train, 10)

y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Convolution2D(20, (5,5), activation = 'relu', input_shape = (28, 28, 1)))
model.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))

model.add(keras.layers.Convolution2D(50,(5,5), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(200, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(
    loss = keras.losses.categorical_crossentropy,
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, 
                   epochs = 12 , 
                   verbose = True,
                    batch_size= 32,
                   validation_split= 0.95)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
pred = model.predict(test_array)

In [ ]:
sub = pd.DataFrame()
sub['ImageId'] = np.arange(1 , len(test)+1, 1)
sub['Label'] = np.argmax(pred, axis = 1)
sub.to_csv('./Submission.csv', index = False)